# Data Cleaning and Preparation for Survey Responses

### 1: Remove Unnecessary Columns
- The `Timestamp` column, which is not necessary for the analysis, is dropped to simplify the dataset.

### 2: Handle Missing Values
- Placeholder values such as 'NIL', 'Na', 'n/a', empty strings, and similar variations are replaced with `NaN` to mark missing data consistently throughout the dataset.

### 3: Standardize Text Data
- All column names are converted to lowercase and stripped of leading/trailing spaces to ensure consistency in comparisons.

### 4: Convert MCQ Columns to Categorical Data Types
- All 37 multiple-choice question (MCQ) columns, such as age, gender, occupation, and ticket type, are converted to the `category` data type. This allows for faster operations like grouping and filtering on categorical data.

### 5: Combine Free-Response Questions
- The three open-ended questions are merged into a single column titled **"Additional Comments or Suggestions"** for simplification and easier analysis.

### 6: Text Cleaning for Open-Ended Responses
- The free-text responses are further cleaned by:
    - Removing non-alphabetic characters (punctuation, numbers).
    - Removing stopwords (both standard and custom stopwords such as 'na', 'nil', etc.).
    - Applying lemmatization to reduce words to their base form, ensuring consistency in text analysis.

### 7: Handling Inconsistent Responses
- Certain survey responses are cleaned to ensure consistency between related questions:
    - For the question **"Did you use an Express Pass?"**, if the response is **"No"**, any answers to the follow-up question **"If your answer was 'yes', how satisfied were you with the value of the Express Pass?"** are set to `NaN`.
    - For the question **"Did you purchase from any of the shopping outlets during your visit?"**, if the response is **"No"**, any values provided for **"How much did you approximately spend on merchandise?"** are also set to `NaN`.

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure necessary NLTK data is downloaded (if not already)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liyuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/liyuan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
file_path = '/Users/liyuan/Desktop/DSA3101 Project Data/Survey Response Data/Real_Survey_Response.csv'
df = pd.read_csv(file_path)

In [3]:
df.head()

,Timestamp,What is your age?,What is your gender?,What is your occupation?,Are you a...?,How did you purchase your tickets?,What type of ticket did you purchase?,Did you purchase your tickets for any special event (e.g. Halloween Horror Night)?,How would you rate the ease of purchasing tickets?,What type of visitor are you?,...,"How satisfied are you with the overall service of the staff? (e.g. friendliness, helpfulness)","How would you rate the cleanliness of the park facilities (e.g. restrooms, common areas)?",Did you find enough seating areas throughout the park?,How satisfied were you with the accessibility of restrooms throughout the park?,What could be improved about the park’s facilities?,"Overall, how satisfied were you with your visit to Universal Studios Singapore?",Which aspect of your visit was the most disappointing? (Select all that apply),Are you likely to return to the park in the future?,Would you recommend Universal Studios Singapore to a friend or family member?,What could Universal Studios Singapore improve to make your visit better?
0,06/10/2024 18:00:00,18 - 25,Male,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",Regular ticket with Express Pass,No,4,Group,...,4,4,Yes,4,Install more water coolers given Singapore’s h...,4,"Long wait times, Overcrowded attractions",Maybe,Yes,NIL
1,06/10/2024 18:09:47,18 - 25,Female,Employed,Singaporean,Resort World Sentosa Website,One-day regular ticket,No,4,Group,...,5,5,Yes,5,Na,5,NaN,Yes,Yes,Na
2,06/10/2024 18:26:43,18 - 25,Female,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",One-day regular ticket,No,5,Group,...,3,3,No,2,More benches and shades around the park,4,"Long wait times, Poor weather conditions, Over...",Yes,Yes,Better crowd management. Live queue to see how...
3,06/10/2024 19:23:49,18 - 25,Male,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",One-day regular ticket,Yes,3,Group,...,4,4,Yes,4,na,4,Long wait times,Yes,Yes,na
4,06/10/2024 20:15:35,18 - 25,Male,Student,Singaporean,Resort World Sentosa Website,One-day regular ticket,No,5,Group,...,4,4,No,4,"sheltered areas, aircon etc.",3,"Long wait times, Overcrowded attractions",Maybe,Yes,as said previously


In [4]:
# 1: Drop unnecessary columns like 'Timestamp'
cleaned_data = df.drop(columns=['Timestamp'])

# 2: Replace placeholders for missing values with NaN
cleaned_data = cleaned_data.replace(['NIL', 'Na', 'na', 'N/A', 'n/a', ''], pd.NA)

# 3: Standardize column names (remove leading/trailing spaces, lowercase)
cleaned_data.columns = cleaned_data.columns.str.strip().str.lower()

In [5]:
# 4: Convert MCQ Columns to Categorical Data Types
mcq_columns = [
    'what is your age?', 'what is your gender?', 'what is your occupation?', 
    'are you a...?', 'how did you purchase your tickets?', 
    'what type of ticket did you purchase?', 
    'did you purchase your tickets for any special event (e.g. halloween horror night)?', 
    'how would you rate the ease of purchasing tickets?', 'what type of visitor are you?', 
    'what month did you visit uss?', 'which day did you visit uss?', 
    'how would you rate your arriving experience?', 
    'how long did you queue at the entrance before entering uss? (e.g. get through security and ticket scanning)', 
    'were staff at the entrance helpful and courteous?', 
    'how would you rate the wait time for the attractions you visited?', 
    'did you use an express pass?', 
    'if your answer was "yes", how satisfied were you with the value of the express pass?', 
    'how satisfied are you with the service of the staff at the attractions?', 
    'how satisfied were you with the overall experience of the attractions?', 
    'did you purchase from any of the shopping outlets during your visit?', 
    'how much did you approximately spend on merchandise?', 
    'was the pricing reasonable for the quality and trademark of the merchandises?', 
    'how would you rate the variety of merchandises?', 
    'did you dine in or purchased from any of the food outlets during your visit?', 
    'how would you rate the variety of food options available?', 
    'how would you rate the overall food quality?', 
    'do you agree that the pricing is reasonable for the food quality?', 
    'how much did you approximately spend on food?', 
    'how long did you wait to get served at the food outlet?', 
    'did you encounter any issues during your visit, and were they resolved to your satisfaction?', 
    'how satisfied are you with the overall service of the staff? (e.g. friendliness, helpfulness)', 
    'how would you rate the cleanliness of the park facilities (e.g. restrooms, common areas)?', 
    'did you find enough seating areas throughout the park?', 
    'how satisfied were you with the accessibility of restrooms throughout the park?', 
    'overall, how satisfied were you with your visit to universal studios singapore?', 
    'are you likely to return to the park in the future?', 
    'would you recommend universal studios singapore to a friend or family member?'
]

# Convert the specified columns to 'category' data type
cleaned_data[mcq_columns] = cleaned_data[mcq_columns].astype('category')

In [6]:
# 5: Combine the three free-response questions into one column
cleaned_data['additional comments or suggestions'] = cleaned_data[
    ['what could have made your ride experience better?', 
     'what could be improved about the park’s facilities?', 
     'what could universal studios singapore improve to make your visit better?']
].fillna('').agg(' '.join, axis=1).str.strip()

# Drop the original free-response columns after combining
cleaned_data = cleaned_data.drop(columns=[
    'what could have made your ride experience better?', 
    'what could be improved about the park’s facilities?', 
    'what could universal studios singapore improve to make your visit better?'
])

In [7]:
# 6: Cleaning and Lemmatizing Open-Ended Text Responses
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
custom_stopwords = {'na', 'nil', 'n/a', 'none', 'no', 'nothing', '---', '--', '-', 'nil nil', 'nil -'}  
stop_words.update(custom_stopwords)

def clean_text(text):
    # Remove non-alphabet characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    # Convert to lowercase
    text = text.lower()  
    # Split the text into individual words
    words = text.split()  
    # Remove stopwords and lemmatize each word
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  
    # Rejoin words into cleaned text
    return " ".join(words)
    
cleaned_data['additional comments or suggestions'] = cleaned_data['additional comments or suggestions'].apply(lambda x: clean_text(str(x)) if pd.notnull(x) else x)

In [8]:
# 7: Handling Inconsistent Responses
# Column names for Express Pass and Shopping Questions
express_pass_column = 'did you use an express pass?'
express_satisfaction_column = 'if your answer was "yes", how satisfied were you with the value of the express pass?'

shopping_column = 'did you purchase from any of the shopping outlets during your visit?'
merchandise_spent_column = 'how much did you approximately spend on merchandise?'

# For Express Pass Satisfaction
# Set satisfaction rating to NaN if the respondent answered "No" to using an Express Pass
cleaned_data.loc[cleaned_data[express_pass_column] == 'No', express_satisfaction_column] = pd.NA

# For Shopping Outlets
# Set merchandise spent value to NaN if the respondent answered "No" to purchasing from shopping outlets
cleaned_data.loc[cleaned_data[shopping_column] == 'No', merchandise_spent_column] = pd.NA

## Column Renaming Documentation

In this section, we detail the changes made to the original survey question column names, simplifying them into shorter, more manageable versions for easier analysis.

| Original Survey Question (Proper Capitalization)                                       | New Column Name                |
|----------------------------------------------------------------------------------------|--------------------------------|
| What is your age?                                                                      | age                            |
| What is your gender?                                                                   | gender                         |
| What is your occupation?                                                               | occupation                     |
| Are you a...?                                                                          | visitor_profile                   |
| How did you purchase your tickets?                                                     | ticket_purchase_method         |
| What type of ticket did you purchase?                                                  | ticket_type                    |
| Did you purchase your tickets for any special event (e.g. Halloween Horror Night)?     | special_event_ticket           |
| How would you rate the ease of purchasing tickets?                                     | ticket_purchase_ease           |
| What type of visitor are you?                                                          | visitor_type                   |
| What month did you visit USS?                                                          | visit_month                    |
| Which day did you visit USS?                                                           | visit_day                      |
| How would you rate your arriving experience?                                           | arrival_experience             |
| How long did you queue at the entrance before entering USS? (e.g. security, scanning)  | queue_time_at_entry            |
| Were staff at the entrance helpful and courteous?                                      | entry_staff_service            |
| How would you rate the wait time for the attractions you visited?                      | attraction_wait_time           |
| Did you use an Express Pass?                                                           | used_express_pass              |
| If your answer was "Yes", how satisfied were you with the value of the Express Pass?   | express_pass_satisfaction      |
| How satisfied are you with the service of the staff at the attractions?                | attraction_staff_service       |
| How satisfied were you with the overall experience of the attractions?                 | attraction_overall_experience  |
| Did you purchase from any of the shopping outlets during your visit?                   | shopping_purchase              |
| How much did you approximately spend on merchandise?                                   | merchandise_spend              |
| Was the pricing reasonable for the quality and trademark of the merchandises?          | merchandise_price_reasonable   |
| How would you rate the variety of merchandises?                                        | merchandise_variety            |
| Did you dine in or purchase from any of the food outlets during your visit?            | food_purchase                  |
| How would you rate the variety of food options available?                              | food_variety                   |
| How would you rate the overall food quality?                                           | food_quality                   |
| Do you agree that the pricing is reasonable for the food quality?                      | food_price_reasonable          |
| How much did you approximately spend on food?                                          | food_spend                     |
| How long did you wait to get served at the food outlet?                                | food_wait_time                 |
| Did you encounter any issues during your visit, and were they resolved to your satisfaction? | visit_issues_resolved     |
| How satisfied are you with the overall service of the staff? (e.g., friendliness, helpfulness) | overall_staff_service     |
| How would you rate the cleanliness of the park facilities (e.g., restrooms, common areas)? | park_cleanliness           |
| Did you find enough seating areas throughout the park?                                 | park_seating_availability      |
| How satisfied were you with the accessibility of restrooms throughout the park?        | restroom_accessibility         |
| Overall, how satisfied were you with your visit to Universal Studios Singapore?        | overall_satisfaction           |
| Are you likely to return to the park in the future?                                    | return_likelihood              |
| Would you recommend Universal Studios Singapore to a friend or family member?          | recommendation_likelihood      |
| Additional comments or suggestions                                                     | additional_comments            |
| Cleaned additional comments                                                            | cleaned_comments               |


In [14]:
# Dictionary to map original column names to shorter versions
column_rename_map = {
    'what is your age?': 'age',
    'what is your gender?': 'gender',
    'what is your occupation?': 'occupation',
    'are you a...?': 'visitor_profile',
    'how did you purchase your tickets?': 'ticket_purchase_method',
    'what type of ticket did you purchase?': 'ticket_type',
    'did you purchase your tickets for any special event (e.g. halloween horror night)?': 'special_event_ticket',
    'how would you rate the ease of purchasing tickets?': 'ticket_purchase_ease',
    'what type of visitor are you?': 'visitor_type',
    'what month did you visit uss?': 'visit_month',
    'which day did you visit uss?': 'visit_day',
    'how would you rate your arriving experience?': 'arrival_experience',
    'how long did you queue at the entrance before entering uss? (e.g. get through security and ticket scanning)': 'queue_time_at_entry',
    'were staff at the entrance helpful and courteous?': 'entry_staff_service',
    'how would you rate the wait time for the attractions you visited?': 'attraction_wait_time',
    'did you use an express pass?': 'used_express_pass',
    'if your answer was "yes", how satisfied were you with the value of the express pass?': 'express_pass_satisfaction',
    'how satisfied are you with the service of the staff at the attractions?': 'attraction_staff_service',
    'how satisfied were you with the overall experience of the attractions?': 'attraction_overall_experience',
    'did you purchase from any of the shopping outlets during your visit?': 'shopping_purchase',
    'how much did you approximately spend on merchandise?': 'merchandise_spend',
    'was the pricing reasonable for the quality and trademark of the merchandises?': 'merchandise_price_reasonable',
    'how would you rate the variety of merchandises?': 'merchandise_variety',
    'did you dine in or purchased from any of the food outlets during your visit?': 'food_purchase',
    'how would you rate the variety of food options available?': 'food_variety',
    'how would you rate the overall food quality?': 'food_quality',
    'do you agree that the pricing is reasonable for the food quality?': 'food_price_reasonable',
    'how much did you approximately spend on food?': 'food_spend',
    'how long did you wait to get served at the food outlet?': 'food_wait_time',
    'did you encounter any issues during your visit, and were they resolved to your satisfaction?': 'visit_issues_resolved',
    'how satisfied are you with the overall service of the staff? (e.g. friendliness, helpfulness)': 'overall_staff_service',
    'how would you rate the cleanliness of the park facilities (e.g. restrooms, common areas)?': 'park_cleanliness',
    'did you find enough seating areas throughout the park?': 'park_seating_availability',
    'how satisfied were you with the accessibility of restrooms throughout the park?': 'restroom_accessibility',
    'overall, how satisfied were you with your visit to universal studios singapore?': 'overall_satisfaction',
    'are you likely to return to the park in the future?': 'return_likelihood',
    'would you recommend universal studios singapore to a friend or family member?': 'recommendation_likelihood',
    'additional comments or suggestions': 'additional_comments',
    'cleaned_additional_comments': 'cleaned_comments'
}

# Rename the columns in the dataset
survey_data_cleaned = cleaned_data.rename(columns=column_rename_map)

In [15]:
survey_data_cleaned.head()

,age,gender,occupation,visitor_profile,ticket_purchase_method,ticket_type,special_event_ticket,ticket_purchase_ease,visitor_type,what are your preferred attraction(s)? select all that apply.,...,visit_issues_resolved,overall_staff_service,park_cleanliness,park_seating_availability,restroom_accessibility,overall_satisfaction,which aspect of your visit was the most disappointing? (select all that apply),return_likelihood,recommendation_likelihood,additional_comments
0,18 - 25,Male,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",Regular ticket with Express Pass,No,4,Group,"Thrill Rides, Shows",...,No issues,4,4,Yes,4,4,"Long wait times, Overcrowded attractions",Maybe,Yes,install water cooler given singapore humid cli...
1,18 - 25,Female,Employed,Singaporean,Resort World Sentosa Website,One-day regular ticket,No,4,Group,"Thrill Rides, Special Events (e.g. Halloween H...",...,No issues,5,5,Yes,5,5,NaN,Yes,Yes,everything gd
2,18 - 25,Female,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",One-day regular ticket,No,5,Group,Thrill Rides,...,No issues,3,3,No,2,4,"Long wait times, Poor weather conditions, Over...",Yes,Yes,shorter waiting time bench shade around park b...
3,18 - 25,Male,Student,Singaporean,"Third Party Vendor (e.g. Klook, Trip.com)",One-day regular ticket,Yes,3,Group,"Thrill Rides, Shows",...,No issues,4,4,Yes,4,4,Long wait times,Yes,Yes,shorter waiting time
4,18 - 25,Male,Student,Singaporean,Resort World Sentosa Website,One-day regular ticket,No,5,Group,"Thrill Rides, Shows",...,No issues,4,4,No,4,3,"Long wait times, Overcrowded attractions",Maybe,Yes,maybe put aircon waitingqueuing area sheltered...


In [16]:
# Save as csv file
survey_data_cleaned.to_csv('survey_data_cleaned.csv', index=False)